# Lyman-alpha emission reconstruction

In this notebook, we will learn how to use `rocky-worlds-utils` to reconstruct the intrinsic Lyman-alpha (Lya) emission of the star GJ 3929.

There is an extensive literature discussing various aspects of the Lya emission of stars (see [Youngblood et al. 2016](https://ui.adsabs.harvard.edu/abs/2016ApJ...824..101Y/abstract) and references therein). In summary, the observed Lya spectra of stars that are not the Sun is significantly absorbed by neutral hydrogen in the interstellar medium (ISM). Thus, in order to estimate the unobstructed, or intrinsic emission, we need to model the line. The modeling takes many assumption, which we shall not discuss extensively here.

- Author: Leonardo Dos Santos (STScI)
- Date first published: November 19, 2025

## Imports

- `astropy`: FITS files I/O
- `matplotlib`: Plotting library
- `numpy`: array calculations
- `rocky_worlds_utils`: various functions to facilitate data analysis

In [ ]:
import astropy.units as u
import astropy.constants as c
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import numpy as np
from rocky_worlds_utils.hst import lya, spectrum

%matplotlib inline
pylab.rcParams['figure.figsize'] = 15.0,9.0
pylab.rcParams['font.size'] = 14

Ideally, we aim to perform the reconstruction from an observation with the highest possible signal-to-noise ratio (SNR). From the previous notebooks, we have two exposures that cover the Lya emission of GJ 3929. In order to increase the SNR of the data, we will co-add these two exposures using the `spectrum.coadd_first_order()` function.

In [ ]:
prefix = './example_data/'
datasets = ['ofkb08020', 'ofkb08030']
n_orbits = len(datasets)

y_plot_scale = 1E-14
# Co-addition is calculated using the following line of code
co_add_spec = spectrum.coadd_first_order(datasets, prefix)
plt.errorbar(co_add_spec['wavelength'], 
             co_add_spec['flux'] / y_plot_scale, 
             yerr=co_add_spec['uncertainty'] / y_plot_scale, 
             fmt='o', color='C3', 
             lw=2, mec='white')

# It is also useful to identify the region with strong geocoronal contamination
# From a visual inspection of the gross counts in comparison with net counts, we
# identify the wavelength range with strong contamination as [1215.3, 1215.8] Angstrom
# Let's block this range with a white span
plt.axvspan(xmin=1215.3, xmax=1215.8, color='white', zorder=3)

plt.xlabel(r'Wavelength [${\rm \AA}$]')
plt.ylabel(r'Flux density [$\times 10^{%i}$ erg s$^{-1}$ cm$^{-2}$ ${\rm \AA}^{-1}$]' % int((np.log10(y_plot_scale))))
plt.xlim(1210, 1220)
plt.ylim(-0.1, 2.1)

With that done, we can proceed with a simple fit "by eye" to the observed spectrum, which will serve as our first guess. 

The stellar instrinsic Lya line is modeled with a Voigt profile with a self-absorption and is defined by the following parameters:
- Amplitude of the Lorentzian component
- Width of the Lorentzian component
- Width of the Gaussian component
- Radial velocity
- Self-absorption parameter

The ISM absorption is modeled as one cloud of hydrogen gas defined by the following parameters:
- Column density of neutral hydrogen
- Gas temperature
- Turbulence velocity
- ISM cloud line-of-sight velocity

See the documentation of the `lya` module for more details on units, definitions and reasonable value ranges. For high SNR data, one might need to model the ISM absorption with multiple clouds, but the RWDDT targets are usually FUV faint so we will keep to only one cloud for this example. Furthermore, we shall assume that there is no self-absorption and negligible ISM turbulence velocity.

In [ ]:
# We shall model wavelengths between 1210 and 1220 only, in an array length of 999.
wavelength = np.linspace(1210, 1220, 999)

# Stellar parameters
star_velocity = 10.14  # km/s
log_lorentzian_amplitude = -14.1  # flam
lorentzian_width = 50  # km/s
gaussian_width = 150  # km/s
self_absorption_parameter = 0

# ISM parameters
n_hi = 18.35  # cm^-2
temp = 2000  # K
ism_los_velocity = -25. # km/s
ism_turb_velocity = 0

# Calculate models for the observed Lya profile (convolved with the instrumental line spread function, or LSF),
# the observable Lya profile (without convolving with the LSF), the intrinsic profile (without ISM absorption)
# and the ISM absorption profile
lya_profile, observable_lya_profile, intrinsic_profile, ism_absorption_profile = lya.observed_lya_profile(
    grating='G140M', 
    aperture='52x0.2', 
    wavelength=wavelength,
    star_log_lorentzian_amplitude=log_lorentzian_amplitude, 
    star_lorentzian_width=lorentzian_width,
    star_gaussian_width=gaussian_width, 
    ism_log_h1_column_density=n_hi,
    ism_gas_temperature=temp, 
    star_velocity=star_velocity,
    star_self_absorption_parameter=self_absorption_parameter,
    ism_turbulence_velocity=ism_turb_velocity, 
    ism_los_velocity=ism_los_velocity,
    return_all_profiles=True
)

plt.plot(wavelength, lya_profile / y_plot_scale, color='k', lw=3, alpha=0.7, label='Observation model', zorder=4)
plt.plot(wavelength, intrinsic_profile / y_plot_scale, color='k', ls='--', lw=3, alpha=0.7, label='Intrinsic Lya profile', zorder=4)
plt.errorbar(co_add_spec['wavelength'], 
             co_add_spec['flux'] / y_plot_scale, 
             yerr=co_add_spec['uncertainty'] / y_plot_scale, 
             fmt='o', color='C3', 
             lw=2, mec='white')

plt.xlabel(r'Wavelength [${\rm \AA}$]')
plt.ylabel(r'Flux density [$\times 10^{%i}$ erg s$^{-1}$ cm$^{-2}$ ${\rm \AA}^{-1}$]' % int((np.log10(y_plot_scale))))
plt.xlim(1210, 1220)
plt.ylim(-0.1, 2.1)
plt.legend()
plt.title('GJ 3929, "by-eye" fit')
plt.tight_layout()